In [125]:
import cv2
from ultralytics import YOLO
import numpy as np
import time
import os
import csv
import torch
import sys
from matplotlib import pyplot as plt

In [126]:
model_path = "runs/detect/train4/weights/best.pt"
recognition_model = YOLO(model_path)

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.device("cuda"))
device = torch.device("cuda")
recognition_model.to(device)

True
0
NVIDIA GeForce GTX 1080 Ti
cuda


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [127]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    - box1: Tuple or List representing (x1, y1, x2, y2) of the first box.
    - box2: Tuple or List representing (x1, y1, x2, y2) of the second box.

    Returns:
    - IoU: Intersection over Union.
    """

    # Extract coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Calculate intersection coordinates
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))

    # Calculate areas of boxes and intersection
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    area_intersection = x_intersection * y_intersection

    # Calculate Union
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    iou = area_intersection / (area_union + 1e-10)  # Add a small epsilon to avoid division by zero

    return iou

# Example usage:
box1 = (0, 0, 4, 4)
box2 = (2, 2, 6, 6)

iou = calculate_iou(box1, box2)
print(f"IoU: {iou}")


IoU: 0.14285714285663267


## testing whitout preprocessing

In [128]:

overal_digits = 0
overal_predicted_digits = 0
overal_correct_digits = 0
overal_numbers = 0
overal_correct_numbers = 0
with open('test_results_no_preprocess.csv', 'w', newline='') as csv_file:

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    rep = 0
    dummy_input = torch.randn(1, 3,224,224, dtype=torch.float).to(device)
    #GPU-WARM-UP
    for _ in range(10):
        _ = recognition_model(dummy_input)
    timings = []

    
    writer = csv.writer(csv_file)
    field = ["filename", "backnumber", "amount_of_numbers","digits_predicted", "number_predicted_correct", "number_of_predicted_digits", "inference_time"]
    writer.writerow(field)
    progress = 0
    for file in os.listdir(os.path.join('datasets','test', 'images')):

        
        img = cv2.imread(os.path.join('datasets','test','images', file))
        starter.record()
        results = recognition_model(img, verbose=False)
        ender.record()

        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings.append(curr_time)
        predicted_digits = []
        average_x = []
        result = results[0].boxes
        digits_to_discard = []
        boxes = []
        for i in range(len(result.cls)):
            for j in range(i+1, len(result.cls)):
                box1 = (result.xyxy[i][0].item(), result.xyxy[i][1].item(), result.xyxy[i][2].item(), result.xyxy[i][3].item())
                box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                iou = calculate_iou(box1, box2)
                if iou > 0.5:
                    if result.conf[i].item() > result.conf[j].item():
                        digits_to_discard.append(j)
                    else:
                        digits_to_discard.append(i)
                    
        for j in range(len(result.cls)):
            if j not in digits_to_discard:
                predicted_digits.append(int(result.cls[j].item()))
                boxes.append( (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item()))
                average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
            
        predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
        predicted_boxes = [x for _,x in sorted(zip(average_x,boxes))]
        predicted_number ="".join([str(i) for i in predicted_digits])
        
        filename = file.split('.')[0] +'.' + file.split('.')[1] +'.txt'
        existing_filepath = os.path.join('datasets','test','labels', filename)
        if os.path.exists(existing_filepath): 
            # Using readlines()
            file1 = open(existing_filepath, 'r')
            Lines = file1.readlines()
            boxes = []
            count = 0
            number = ''
            digits = []
            average_x = []
            height, width, _ = img.shape
            # Strips the newline character
            for line in Lines:
                count += 1
                digits.append(line.split(' ')[0])
                average_x.append(line.split(' ')[1])
                box = ((float(line.split(' ')[1]) - float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) - float(line.split(' ')[4])/2)*height, (float(line.split(' ')[1]) + float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) + float(line.split(' ')[4])/2)*height)
                boxes.append(box)
            digits = [x for _, x in sorted(zip(average_x, digits))]
            boxes = [x for _, x in sorted(zip(average_x, boxes))]
            number ="".join([str(i) for i in digits])
            correctly_predicted_digits = 0
            already_found = []
            for i in range(len(predicted_digits)):
                for j in range(len(digits)):
                    if j not in already_found:
                        if calculate_iou(predicted_boxes[i], boxes[j]) >0.5:
                            if predicted_digits[i] == int(digits[j]):
                                correctly_predicted_digits += 1
                                already_found.append(j)

            overal_digits+= count
            overal_numbers+= 1
            overal_predicted_digits += len(predicted_digits)
            overal_correct_digits += correctly_predicted_digits
            if number == predicted_number:
                overal_correct_numbers += 1
            field = [filename, number, str(count), correctly_predicted_digits,number == predicted_number, predicted_digits, curr_time]
            writer.writerow(field)
            progress+=1
        if progress % 500 == 0:
            print(progress)
        """if progress >1:
            break"""
        


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.469890594482422. Dividing input by 255.
0: 224x224 (no detections), 18.7ms
Speed: 0.1ms preprocess, 18.7ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.469890594482422. Dividing input by 255.
0: 224x224 (no detections), 15.4ms
Speed: 0.1ms preprocess, 15.4ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.469890594482422. Dividing input by 255.
0: 224x224 (no detections), 15.6ms
Speed: 0.1ms preprocess, 15.6ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.469890594482422. Dividing input by 255.
0: 224x224 (no detections), 15.2ms
Speed: 0.1ms preprocess, 15.2ms inference, 1.2ms postprocess per image at shape (1, 3, 22

In [129]:
print(f'the total number of digits in the testing dataset: {overal_digits}')
print(f'the number of digits that where predicted: {overal_predicted_digits}')
print(f'the digits that were predicted correctly: {overal_correct_digits}')
print(f'the amount of numbers that were evaluated: {overal_numbers}')
print(f'the numbers that werepredicted correct entirely: {overal_correct_numbers}')
print(f'average inference time: {np.sum(np.array(timings))/len(timings)}')

the total number of digits in the testing dataset: 13920
the number of digits that where predicted: 11915
the digits that were predicted correctly: 10082
the amount of numbers that were evaluated: 6000
the numbers that werepredicted correct entirely: 3284
average inference time: 19.62333707300822


## testing with preprocessing, watershed based, perspective transform

In [130]:
import cv2 as cv

In [131]:
def get_contours_thresh(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,217,255,cv.THRESH_BINARY)
    
    
    # noise removal
    kernel = np.ones((1,1),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 1)
    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)
    canvas = np.zeros((img.shape[0],img.shape[1],3), np.uint8)
    contours, hierarchy = cv.findContours(unknown, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    if len(contours) != 0:
        cnt = sorted(contours, key=cv.contourArea, reverse=True)[0]
        cv.drawContours(canvas, cnt, -1, (0, 255, 255))
        return cnt
    return contours

In [132]:
def get_corners(cnt):
    
    def find_closest(edge):
        min_dist = np.inf
        closest = [0,0]
        for point in cnt:
            dist = (edge[0]-point[0][0])**2+(edge[1]-point[0][1])**2
            if dist<min_dist:
                min_dist = dist
                closest = point[0]
        return closest
            
    def order_corner_points(corners):
        # Separate corners into individual points
        # Index 0 - top-right
        #       1 - top-left
        #       2 - bottom-left
        #       3 - bottom-right
        nr = len(corners)
        corners = np.array([[corner[0][0], corner[0][1]] for corner in corners])
        average_x = corners[:,0].sum()/nr
        average_y =  corners[:,1].sum()/nr
        top_l, top_r, bottom_r, bottom_l = [None, None], [None, None],[None, None],[None, None]
        for corner in corners:
            if corner[1]<average_y:
                if corner[0] < average_x:
                    top_l = corner
                else:
                    top_r = corner
            else:
                if corner[0] < average_x:
                    bottom_l = corner
                else:
                    bottom_r = corner
        if top_l[0] == None:
            #find point closest to [0,0]
            top_l = find_closest([0,0])
        if top_r[0] == None:
            #find point closest to [cnt.shape[1]-1,0]
            top_r = find_closest([original.shape[1]-1,0])
        if bottom_l[0] == None:
            #find point closest to [0,cnt.shape[0]-1]
            bottom_l= find_closest([0,original.shape[0]-1])
        if bottom_r[0] == None:
            #find point closest to [cnt.shape[1]-1,cnt.shape[0]-1]
            bottom_r= find_closest([original.shape[1]-1,original.shape[0]-1])
        top_l[1] = top_l[1]-10 if top_l[1]-10>0 else 0
        top_r[1] = top_r[1]-10 if top_r[1]-10>0 else 0
        top_l[0] = top_l[0]-10 if top_l[0]-10>0 else 0
        top_r[0] = top_r[0]+10 if top_r[0]+10 < original.shape[1] else original.shape[1]-1
        return (top_l, top_r, bottom_r, bottom_l)
    
    
    epsilon = 0.06 * cv.arcLength(cnt, True)
    approx_corners = cv.approxPolyDP(cnt, epsilon, True)
    corners = order_corner_points(approx_corners)
    canvas = np.zeros((original.shape[0],original.shape[1],3), np.uint8)
    cv.drawContours(canvas, cnt, -1, (0, 255, 255))


    
    return corners

In [133]:
def perspective_transform(image, corners):

    
    top_l, top_r, bottom_r, bottom_l = corners
    
    ordered_corners = (top_l, top_r, bottom_r, bottom_l)

    # Determine width of new image which is the max distance between 
    # (bottom right and bottom left) or (top right and top left) x-coordinates
    width_A = np.sqrt(((bottom_r[0] - bottom_l[0]) ** 2) + ((bottom_r[1] - bottom_l[1]) ** 2))
    width_B = np.sqrt(((top_r[0] - top_l[0]) ** 2) + ((top_r[1] - top_l[1]) ** 2))
    width = max(int(width_A), int(width_B))

    # Determine height of new image which is the max distance between 
    # (top right and bottom right) or (top left and bottom left) y-coordinates
    height_A = np.sqrt(((top_r[0] - bottom_r[0]) ** 2) + ((top_r[1] - bottom_r[1]) ** 2))
    height_B = np.sqrt(((top_l[0] - bottom_l[0]) ** 2) + ((top_l[1] - bottom_l[1]) ** 2))
    height = max(int(height_A), int(height_B))

    # Construct new points to obtain top-down view of image in 
    # top_r, top_l, bottom_l, bottom_r order
    dimensions = np.array([[0, 0], [width - 1,0], [width - 1 , height - 1], 
                    [0,height - 1]], dtype = "float32")

    # Convert to Numpy format
    ordered_corners = np.array(ordered_corners, dtype="float32")

    # Find perspective transform matrix
    matrix = cv.getPerspectiveTransform(ordered_corners, dimensions)
    # Transform the image
    transformed = cv.warpPerspective(image, matrix, (width, height),flags=cv.INTER_NEAREST,borderMode=cv.BORDER_REPLICATE)
    
    
    return transformed

In [134]:

overal_digits = 0
overal_predicted_digits = 0
overal_correct_digits = 0
overal_numbers = 0
overal_correct_numbers = 0
with open('test_results_watershed_perspective.csv', 'w', newline='') as csv_file:

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    rep = 0
    dummy_input = torch.randn(1, 3,224,224, dtype=torch.float).to(device)
    #GPU-WARM-UP
    for _ in range(10):
        _ = recognition_model(dummy_input)
    timings = []

    
    writer = csv.writer(csv_file)
    field = ["filename", "backnumber", "amount_of_numbers","digits_predicted", "number_predicted_correct", "number_of_predicted_digits", "inference_time"]
    writer.writerow(field)
    progress = 0
    for file in os.listdir(os.path.join('datasets','test', 'images')):

        
        img = cv2.imread(os.path.join('datasets','test','images', file))
        original = np.copy(img)
        img_copy = np.copy(img)
        starter.record()
        cnt = get_contours_thresh(img_copy)
        if len(cnt)!= 0:
            corners = get_corners(cnt)
            if len(corners) <= 4:
                img = perspective_transform(original, corners)
        results = recognition_model(img, verbose=False)
        ender.record()

        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings.append(curr_time)
        predicted_digits = []
        average_x = []
        result = results[0].boxes
        digits_to_discard = []
        boxes = []
        for i in range(len(result.cls)):
            for j in range(i+1, len(result.cls)):
                box1 = (result.xyxy[i][0].item(), result.xyxy[i][1].item(), result.xyxy[i][2].item(), result.xyxy[i][3].item())
                box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                iou = calculate_iou(box1, box2)
                if iou > 0.5:
                    if result.conf[i].item() > result.conf[j].item():
                        digits_to_discard.append(j)
                    else:
                        digits_to_discard.append(i)
                    
        for j in range(len(result.cls)):
            if j not in digits_to_discard:
                predicted_digits.append(int(result.cls[j].item()))
                boxes.append( (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item()))
                average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
            
        predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
        predicted_boxes = [x for _,x in sorted(zip(average_x,boxes))]
        predicted_number ="".join([str(i) for i in predicted_digits])
        
        filename = file.split('.')[0] +'.' + file.split('.')[1] +'.txt'
        existing_filepath = os.path.join('datasets','test','labels', filename)
        if os.path.exists(existing_filepath): 
            # Using readlines()
            file1 = open(existing_filepath, 'r')
            Lines = file1.readlines()
            boxes = []
            count = 0
            number = ''
            digits = []
            average_x = []
            height, width, _ = img.shape
            # Strips the newline character
            for line in Lines:
                count += 1
                digits.append(line.split(' ')[0])
                average_x.append(line.split(' ')[1])
                box = ((float(line.split(' ')[1]) - float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) - float(line.split(' ')[4])/2)*height, (float(line.split(' ')[1]) + float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) + float(line.split(' ')[4])/2)*height)
                boxes.append(box)
            digits = [x for _, x in sorted(zip(average_x, digits))]
            boxes = [x for _, x in sorted(zip(average_x, boxes))]
            number ="".join([str(i) for i in digits])
            correctly_predicted_digits = 0
            """already_found = []
            for i in range(len(predicted_digits)):
                for j in range(len(digits)):
                    if j not in already_found:
                        if calculate_iou(predicted_boxes[i], boxes[j]) >0.5:
                            if predicted_digits[i] == int(digits[j]):
                                correctly_predicted_digits += 1
                                already_found.append(j)"""

            overal_digits+= count
            overal_numbers+= 1
            overal_predicted_digits += len(predicted_digits)
            overal_correct_digits += correctly_predicted_digits
            if number == predicted_number:
                overal_correct_numbers += 1
            field = [filename, number, str(count), correctly_predicted_digits,number == predicted_number, predicted_digits, curr_time]
            writer.writerow(field)
            progress+=1
        if progress % 500 == 0:
            print(progress)
        

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.536978244781494. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normali

In [135]:
print(f'the total number of digits in the testing dataset: {overal_digits}')
print(f'the number of digits that where predicted: {overal_predicted_digits}')
print(f'the digits that were predicted correctly: {overal_correct_digits}')
print(f'the amount of numbers that were evaluated: {overal_numbers}')
print(f'the numbers that werepredicted correct entirely: {overal_correct_numbers}')
print(f'average inference time: {np.sum(np.array(timings))/len(timings)}')

the total number of digits in the testing dataset: 13920
the number of digits that where predicted: 6240
the digits that were predicted correctly: 0
the amount of numbers that were evaluated: 6000
the numbers that werepredicted correct entirely: 1550
average inference time: 22.303172561327617


## now use homography

In [136]:
def unwarp(img, src):
    """
    Args:
        img: np.array
        src: list
        dst: list
    Returns:
        un_warped: np.array
    """
    
    top_l, top_r, bottom_r, bottom_l = src
    
    ordered_corners = (top_l, top_r, bottom_r, bottom_l)

    # Determine width of new image which is the max distance between 
    # (bottom right and bottom left) or (top right and top left) x-coordinates
    width_A = np.sqrt(((bottom_r[0] - bottom_l[0]) ** 2) + ((bottom_r[1] - bottom_l[1]) ** 2))
    width_B = np.sqrt(((top_r[0] - top_l[0]) ** 2) + ((top_r[1] - top_l[1]) ** 2))
    width = max(int(width_A), int(width_B))

    # Determine height of new image which is the max distance between 
    # (top right and bottom right) or (top left and bottom left) y-coordinates
    height_A = np.sqrt(((top_r[0] - bottom_r[0]) ** 2) + ((top_r[1] - bottom_r[1]) ** 2))
    height_B = np.sqrt(((top_l[0] - bottom_l[0]) ** 2) + ((top_l[1] - bottom_l[1]) ** 2))
    height = max(int(height_A), int(height_B))

    # Construct new points to obtain top-down view of image in 
    # top_r, top_l, bottom_l, bottom_r order
    dimensions = np.array([[0, 0], [width - 1,0],[0,height - 1], [width - 1 , height - 1]
                    ], dtype = "float32")
    
    h, w = img.shape[:2]
    H, _ = cv.findHomography(np.array([top_l,  top_r, bottom_l, bottom_r]), dimensions, method=cv.RANSAC, ransacReprojThreshold=3.0)
    
    un_warped = cv.warpPerspective(img, H, (w, h), flags=cv.INTER_LINEAR)

    # plot
    return un_warped


In [137]:

overal_digits = 0
overal_predicted_digits = 0
overal_correct_digits = 0
overal_numbers = 0
overal_correct_numbers = 0
with open('test_results_watershed_homography.csv', 'w', newline='') as csv_file:

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    rep = 0
    dummy_input = torch.randn(1, 3,224,224, dtype=torch.float).to(device)
    #GPU-WARM-UP
    for _ in range(10):
        _ = recognition_model(dummy_input)
    timings = []

    
    writer = csv.writer(csv_file)
    field = ["filename", "backnumber", "amount_of_numbers","digits_predicted", "number_predicted_correct", "number_of_predicted_digits", "inference_time"]
    writer.writerow(field)
    progress = 0
    for file in os.listdir(os.path.join('datasets','test', 'images')):

        
        img = cv2.imread(os.path.join('datasets','test','images', file))
        original = np.copy(img)
        img_copy = np.copy(img)
        starter.record()
        cnt = get_contours_thresh(img_copy)
        if len(cnt)!= 0:
            corners = get_corners(cnt)
            if len(corners) <= 4:
                img = unwarp(original, corners)
        results = recognition_model(img, verbose=False)
        ender.record()

        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings.append(curr_time)
        predicted_digits = []
        average_x = []
        result = results[0].boxes
        digits_to_discard = []
        boxes = []
        for i in range(len(result.cls)):
            for j in range(i+1, len(result.cls)):
                box1 = (result.xyxy[i][0].item(), result.xyxy[i][1].item(), result.xyxy[i][2].item(), result.xyxy[i][3].item())
                box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                iou = calculate_iou(box1, box2)
                if iou > 0.5:
                    if result.conf[i].item() > result.conf[j].item():
                        digits_to_discard.append(j)
                    else:
                        digits_to_discard.append(i)
                    
        for j in range(len(result.cls)):
            if j not in digits_to_discard:
                predicted_digits.append(int(result.cls[j].item()))
                boxes.append( (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item()))
                average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
            
        predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
        predicted_boxes = [x for _,x in sorted(zip(average_x,boxes))]
        predicted_number ="".join([str(i) for i in predicted_digits])

        
        filename = file.split('.')[0] +'.' + file.split('.')[1] +'.txt'
        existing_filepath = os.path.join('datasets','test','labels', filename)
        if os.path.exists(existing_filepath): 
            # Using readlines()
            file1 = open(existing_filepath, 'r')
            Lines = file1.readlines()
            boxes = []
            count = 0
            number = ''
            digits = []
            average_x = []
            height, width, _ = img.shape
            # Strips the newline character
            for line in Lines:
                count += 1
                digits.append(line.split(' ')[0])
                average_x.append(line.split(' ')[1])
                box = ((float(line.split(' ')[1]) - float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) - float(line.split(' ')[4])/2)*height, (float(line.split(' ')[1]) + float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) + float(line.split(' ')[4])/2)*height)
                boxes.append(box)
            digits = [x for _, x in sorted(zip(average_x, digits))]
            boxes = [x for _, x in sorted(zip(average_x, boxes))]
            number ="".join([str(i) for i in digits])
            correctly_predicted_digits = 0
            already_found = []
            """for i in range(len(predicted_digits)):
                for j in range(len(digits)):
                    if j not in already_found:
                        if calculate_iou(predicted_boxes[i], boxes[j]) >0.5:
                            if predicted_digits[i] == int(digits[j]):
                                correctly_predicted_digits += 1
                                already_found.append(j)"""

            overal_digits+= count
            overal_numbers+= 1
            overal_predicted_digits += len(predicted_digits)
            overal_correct_digits += correctly_predicted_digits
            if number == predicted_number:
                overal_correct_numbers += 1
            field = [filename, number, str(count), correctly_predicted_digits,number == predicted_number, predicted_digits, curr_time]
            writer.writerow(field)
            progress+=1

        if progress % 500 == 0:
            print(progress)
        """if progress >1:
            break"""
        

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.292972564697266. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normali

In [138]:
print(f'the total number of digits in the testing dataset: {overal_digits}')
print(f'the number of digits that where predicted: {overal_predicted_digits}')
print(f'the digits that were predicted correctly: {overal_correct_digits}')
print(f'the amount of numbers that were evaluated: {overal_numbers}')
print(f'the numbers that werepredicted correct entirely: {overal_correct_numbers}')
print(f'average inference time: {np.sum(np.array(timings))/len(timings)}')

the total number of digits in the testing dataset: 13920
the number of digits that where predicted: 7170
the digits that were predicted correctly: 0
the amount of numbers that were evaluated: 6000
the numbers that werepredicted correct entirely: 1759
average inference time: 22.622637280464172


## now use kmeans first with perspective
try later to use https://www.blog.dailydoseofds.com/p/make-sklearn-kmeans-20x-times-faster \
or try kmeans gpu

In [139]:

from sklearn.cluster import KMeans

# can initialize kmeans one time so don't add to time
kmeans=KMeans(n_clusters=8, n_init=1)

In [140]:
def get_contours_kmeans(img):
    img=img.reshape((img.shape[1]*img.shape[0],3))
    s=kmeans.fit(img)
    labels=kmeans.labels_
    centroid=kmeans.cluster_centers_
    
    good_labels = []
    for i in range(len(centroid)):
        if centroid[i].max() - centroid[i].min() <35 and centroid[i].min()>155:
            good_labels.append(i)
    mask = np.zeros(len(labels), dtype = np.uint8)
    for i in range(len(labels)):
        if labels[i] in good_labels:
            mask[i] = 255
    mask = mask.reshape((original.shape[0],original.shape[1]))
    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    if len(contours) != 0:
        cnt = sorted(contours, key=cv.contourArea, reverse=True)[0]
        return cnt
    return contours

In [141]:

overal_digits = 0
overal_predicted_digits = 0
overal_correct_digits = 0
overal_numbers = 0
overal_correct_numbers = 0
with open('test_results_kmeans_perspective.csv', 'w', newline='') as csv_file:

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    rep = 0
    dummy_input = torch.randn(1, 3,224,224, dtype=torch.float).to(device)
    #GPU-WARM-UP
    for _ in range(10):
        _ = recognition_model(dummy_input)
    timings = []

    
    writer = csv.writer(csv_file)
    field = ["filename", "backnumber", "amount_of_numbers","digits_predicted", "number_predicted_correct", "number_of_predicted_digits", "inference_time"]
    writer.writerow(field)
    progress = 0
    for file in os.listdir(os.path.join('datasets','test', 'images')):

        
        img = cv2.imread(os.path.join('datasets','test','images', file))
        original = np.copy(img)
        img_copy = np.copy(img)
        starter.record()
        #start_time = time.time()
        cnt = get_contours_kmeans(img_copy)
        if len(cnt)!= 0:
            corners = get_corners(cnt)
            if len(corners) <= 4:
                img = perspective_transform(original, corners)
        
        #print("My program took", time.time() - start_time, "to run")
        results = recognition_model(img, verbose=False)
        ender.record()

        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings.append(curr_time)
        predicted_digits = []
        average_x = []
        result = results[0].boxes
        digits_to_discard = []
        boxes = []
        for i in range(len(result.cls)):
            for j in range(i+1, len(result.cls)):
                box1 = (result.xyxy[i][0].item(), result.xyxy[i][1].item(), result.xyxy[i][2].item(), result.xyxy[i][3].item())
                box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                iou = calculate_iou(box1, box2)
                if iou > 0.5:
                    if result.conf[i].item() > result.conf[j].item():
                        digits_to_discard.append(j)
                    else:
                        digits_to_discard.append(i)
                    
        for j in range(len(result.cls)):
            if j not in digits_to_discard:
                predicted_digits.append(int(result.cls[j].item()))
                boxes.append( (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item()))
                average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
            
        predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
        predicted_boxes = [x for _,x in sorted(zip(average_x,boxes))]
        predicted_number ="".join([str(i) for i in predicted_digits])
        
        filename = file.split('.')[0] +'.' + file.split('.')[1] +'.txt'
        existing_filepath = os.path.join('datasets','test','labels', filename)
        if os.path.exists(existing_filepath): 
            # Using readlines()
            file1 = open(existing_filepath, 'r')
            Lines = file1.readlines()
            boxes = []
            count = 0
            number = ''
            digits = []
            average_x = []
            height, width, _ = img.shape
            # Strips the newline character
            for line in Lines:
                count += 1
                digits.append(line.split(' ')[0])
                average_x.append(line.split(' ')[1])
                box = ((float(line.split(' ')[1]) - float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) - float(line.split(' ')[4])/2)*height, (float(line.split(' ')[1]) + float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) + float(line.split(' ')[4])/2)*height)
                boxes.append(box)
            digits = [x for _, x in sorted(zip(average_x, digits))]
            boxes = [x for _, x in sorted(zip(average_x, boxes))]
            number ="".join([str(i) for i in digits])
            correctly_predicted_digits = 0
            already_found = []
            """for i in range(len(predicted_digits)):
                for j in range(len(digits)):
                    if j not in already_found:
                        if calculate_iou(predicted_boxes[i], boxes[j]) >0.5:
                            if predicted_digits[i] == int(digits[j]):
                                correctly_predicted_digits += 1
                                already_found.append(j)"""

            overal_digits+= count
            overal_numbers+= 1
            overal_predicted_digits += len(predicted_digits)
            overal_correct_digits += correctly_predicted_digits
            if number == predicted_number:
                overal_correct_numbers += 1
            field = [filename, number, str(count), correctly_predicted_digits,number == predicted_number, predicted_digits, curr_time]
            writer.writerow(field)
            progress+=1
        if progress % 500 == 0:
            print(progress)
        

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.81268835067749. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-

In [142]:
print(f'the total number of digits in the testing dataset: {overal_digits}')
print(f'the number of digits that where predicted: {overal_predicted_digits}')
print(f'the digits that were predicted correctly: {overal_correct_digits}')
print(f'the amount of numbers that were evaluated: {overal_numbers}')
print(f'the numbers that werepredicted correct entirely: {overal_correct_numbers}')
print(f'average inference time: {np.sum(np.array(timings))/len(timings)}')

the total number of digits in the testing dataset: 13920
the number of digits that where predicted: 8792
the digits that were predicted correctly: 0
the amount of numbers that were evaluated: 6000
the numbers that werepredicted correct entirely: 2224
average inference time: 117.99039589977265


## now use homography

In [145]:

overal_digits = 0
overal_predicted_digits = 0
overal_correct_digits = 0
overal_numbers = 0
overal_correct_numbers = 0
with open('test_results_kmeans_homography.csv', 'w', newline='') as csv_file:

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    rep = 0
    dummy_input = torch.randn(1, 3,224,224, dtype=torch.float).to(device)
    #GPU-WARM-UP
    for _ in range(10):
        _ = recognition_model(dummy_input)
    timings = []

    
    writer = csv.writer(csv_file)
    field = ["filename", "backnumber", "amount_of_numbers","digits_predicted", "number_predicted_correct", "number_of_predicted_digits", "inference_time"]
    writer.writerow(field)
    progress = 0
    for file in os.listdir(os.path.join('datasets','test', 'images')):

        
        img = cv2.imread(os.path.join('datasets','test','images', file))
        original = np.copy(img)
        img_copy = np.copy(img)
        starter.record()
        #start_time = time.time()
        cnt = get_contours_kmeans(img_copy)
        if len(cnt)!= 0:
            corners = get_corners(cnt)
            if len(corners) <= 4:
                img = unwarp(original, corners)
        #print("My program took", time.time() - start_time, "to run")
        results = recognition_model(img, verbose=False)
        ender.record()

        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings.append(curr_time)
        predicted_digits = []
        average_x = []
        result = results[0].boxes
        digits_to_discard = []
        boxes = []
        for i in range(len(result.cls)):
            for j in range(i+1, len(result.cls)):
                box1 = (result.xyxy[i][0].item(), result.xyxy[i][1].item(), result.xyxy[i][2].item(), result.xyxy[i][3].item())
                box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                iou = calculate_iou(box1, box2)
                if iou > 0.5:
                    if result.conf[i].item() > result.conf[j].item():
                        digits_to_discard.append(j)
                    else:
                        digits_to_discard.append(i)
                    
        for j in range(len(result.cls)):
            if j not in digits_to_discard:
                predicted_digits.append(int(result.cls[j].item()))
                boxes.append( (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item()))
                average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
            
        predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
        predicted_boxes = [x for _,x in sorted(zip(average_x,boxes))]
        predicted_number ="".join([str(i) for i in predicted_digits])
        
        filename = file.split('.')[0] +'.' + file.split('.')[1] +'.txt'
        existing_filepath = os.path.join('datasets','test','labels', filename)
        if os.path.exists(existing_filepath): 
            # Using readlines()
            file1 = open(existing_filepath, 'r')
            Lines = file1.readlines()
            boxes = []
            count = 0
            number = ''
            digits = []
            average_x = []
            height, width, _ = img.shape
            # Strips the newline character
            for line in Lines:
                count += 1
                digits.append(line.split(' ')[0])
                average_x.append(line.split(' ')[1])
                box = ((float(line.split(' ')[1]) - float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) - float(line.split(' ')[4])/2)*height, (float(line.split(' ')[1]) + float(line.split(' ')[3])/2)*width,(float(line.split(' ')[2]) + float(line.split(' ')[4])/2)*height)
                boxes.append(box)
            digits = [x for _, x in sorted(zip(average_x, digits))]
            boxes = [x for _, x in sorted(zip(average_x, boxes))]
            number ="".join([str(i) for i in digits])
            correctly_predicted_digits = 0
            already_found = []
            """for i in range(len(predicted_digits)):
                for j in range(len(digits)):
                    if j not in already_found:
                        if calculate_iou(predicted_boxes[i], boxes[j]) >0.5:
                            if predicted_digits[i] == int(digits[j]):
                                correctly_predicted_digits += 1
                                already_found.append(j)"""

            overal_digits+= count
            overal_numbers+= 1
            overal_predicted_digits += len(predicted_digits)
            overal_correct_digits += correctly_predicted_digits
            if number == predicted_number:
                overal_correct_numbers += 1
            field = [filename, number, str(count), correctly_predicted_digits,number == predicted_number, predicted_digits, curr_time]
            writer.writerow(field)
            progress+=1
        if progress % 500 == 0:
            print(progress)
        

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.424120903015137. Dividing input by 255.
WARNING ⚠️ torch.Tensor inputs should be normali

In [146]:
print(f'the total number of digits in the testing dataset: {overal_digits}')
print(f'the number of digits that where predicted: {overal_predicted_digits}')
print(f'the digits that were predicted correctly: {overal_correct_digits}')
print(f'the amount of numbers that were evaluated: {overal_numbers}')
print(f'the numbers that werepredicted correct entirely: {overal_correct_numbers}')
print(f'average inference time: {np.sum(np.array(timings))/len(timings)}')

the total number of digits in the testing dataset: 13920
the number of digits that where predicted: 9347
the digits that were predicted correctly: 0
the amount of numbers that were evaluated: 6000
the numbers that werepredicted correct entirely: 2278
average inference time: 117.96601511955261
